<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 2*
# Convolutional Neural Networks (CNNs)

# Assignment

- <a href="#p1">Part 1:</a> Pre-Trained Model
- <a href="#p2">Part 2:</a> Custom CNN Model
- <a href="#p3">Part 3:</a> CNN with Data Augmentation


You will apply three different CNN models to a binary image classification model using Keras. Classify images of Mountains (`./data/mountain/*`) and images of forests (`./data/forest/*`). Treat mountains as the postive class (1) and the forest images as the negative (zero). 

|Mountain (+)|Forest (-)|
|---|---|
|![](https://github.com/b-whitman/DS-Unit-4-Sprint-3-Deep-Learning/blob/master/module2-convolutional-neural-networks/data/mountain/art1131.jpg?raw=1)|![](https://github.com/b-whitman/DS-Unit-4-Sprint-3-Deep-Learning/blob/master/module2-convolutional-neural-networks/data/forest/cdmc317.jpg?raw=1)|

The problem is realively difficult given that the sample is tiny: there are about 350 observations per class. This sample size might be something that you can expect with prototyping an image classification problem/solution at work. Get accustomed to evaluating several differnet possible models.

# Pre - Trained Model
<a id="p1"></a>

Load a pretrained network from Keras, [ResNet50](https://tfhub.dev/google/imagenet/resnet_v1_50/classification/1) - a 50 layer deep network trained to recognize [1000 objects](https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt). Starting usage:

```python
import numpy as np

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D()
from tensorflow.keras.models import Model # This is the functional API

resnet = ResNet50(weights='imagenet', include_top=False)

```

The `include_top` parameter in `ResNet50` will remove the full connected layers from the ResNet model. The next step is to turn off the training of the ResNet layers. We want to use the learned parameters without updating them in future training passes. 

```python
for layer in resnet.layers:
    layer.trainable = False
```

Using the Keras functional API, we will need to additional additional full connected layers to our model. We we removed the top layers, we removed all preivous fully connected layers. In other words, we kept only the feature processing portions of our network. You can expert with additional layers beyond what's listed here. The `GlobalAveragePooling2D` layer functions as a really fancy flatten function by taking the average of each of the last convolutional layer outputs (which is two dimensional still). 

```python
x = res.output
x = GlobalAveragePooling2D()(x) # This layer is a really fancy flatten
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(res.input, predictions)
```

Your assignment is to apply the transfer learning above to classify images of Mountains (`./data/mountain/*`) and images of forests (`./data/forest/*`). Treat mountains as the postive class (1) and the forest images as the negative (zero). 

Steps to complete assignment: 
1. Load in Image Data into numpy arrays (`X`) 
2. Create a `y` for the labels
3. Train your model with pretrained layers from resnet
4. Report your model's accuracy

## Load in Data

![skimage-logo](https://scikit-image.org/_static/img/logo.png)

Check out out [`skimage`](https://scikit-image.org/) for useful functions related to processing the images. In particular checkout the documentation for `skimage.io.imread_collection` and `skimage.transform.resize`.

In [0]:
import numpy as np

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
 
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, MaxPooling2D, Conv2D
from tensorflow.keras.models import Sequential, Model # This is the functional API
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
resnet = ResNet50(input_shape=(256, 256, 3),
                  weights='imagenet', 
                  include_top=False)

94773248/94765736 [==============================] - 1s 0us/step


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from skimage.io import imread_collection
from skimage.transform import resize

import numpy as np

from sklearn.model_selection import train_test_split

#your path 
forests = '/content/drive/My Drive/u4s3m2_data/forest/*.jpg'
mountains = '/content/drive/My Drive/u4s3m2_data/mountain/*.jpg'

#creating a collection with the available images
forests = imread_collection(forests).concatenate()
mountains = imread_collection(mountains).concatenate()

y_0 =  np.zeros(forests.shape[0])
y_1 =  np.ones(mountains.shape[0])

In [0]:
X = np.concatenate([forests, mountains])

In [0]:
# X = resize(X, (702,224,224,3))
y = np.concatenate([y_0, y_1])

In [0]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=.5)

In [0]:
for layer in resnet.layers:
    layer.trainable = False

## Instatiate Model

In [0]:
x = resnet.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(resnet.input, predictions)

In [0]:
model.compile(optimizer='adam',
              loss=binary_crossentropy,
              metrics=['accuracy'])

In [17]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 262, 262, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 128, 128, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 128, 128, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

## Fit Model

In [18]:
history = model.fit(x=x_train, y=y_train, epochs=10, validation_data=(x_test, y_test))

Epoch 1/10
11/11 [==============================] - 2s 222ms/step - loss: 0.4495 - accuracy: 0.8604 - val_loss: 0.1881 - val_accuracy: 0.9630
Epoch 2/10
11/11 [==============================] - 1s 128ms/step - loss: 0.0895 - accuracy: 0.9687 - val_loss: 0.0520 - val_accuracy: 0.9886
Epoch 3/10
11/11 [==============================] - 1s 129ms/step - loss: 0.0331 - accuracy: 0.9915 - val_loss: 0.0468 - val_accuracy: 0.9886
Epoch 4/10
11/11 [==============================] - 1s 129ms/step - loss: 0.0018 - accuracy: 1.0000 - val_loss: 0.1142 - val_accuracy: 0.9772
Epoch 5/10
11/11 [==============================] - 1s 128ms/step - loss: 0.0042 - accuracy: 1.0000 - val_loss: 0.0733 - val_accuracy: 0.9801
Epoch 6/10
11/11 [==============================] - 1s 128ms/step - loss: 2.4385e-04 - accuracy: 1.0000 - val_loss: 0.0503 - val_accuracy: 0.9829
Epoch 7/10
11/11 [==============================] - 1s 128ms/step - loss: 2.5659e-04 - accuracy: 1.0000 - val_loss: 0.0473 - val_accuracy: 0.980

Hooray, it worked!

# Custom CNN Model

In this step, write and train your own convolutional neural network using Keras. You can use any architecture that suits you as long as it has at least one convolutional and one pooling layer at the beginning of the network - you can add more if you want. 

In [0]:
my_model = Sequential()
my_model.add(Conv2D(32, kernel_size=(2,2), activation='relu', input_shape=(256,256,3)))
my_model.add(MaxPooling2D(pool_size=(2,2)))
my_model.add(Conv2D(64, kernel_size=(2,2), activation='relu'))
my_model.add(MaxPooling2D(pool_size=(2,2)))
my_model.add(Conv2D(64, kernel_size=(2,2), activation='relu'))
my_model.add(Flatten())
my_model.add(Dense(64, activation='relu'))
my_model.add(Dense(1, activation='sigmoid'))

my_model.compile(optimizer='adam',
                 loss=binary_crossentropy,
                 metrics=['accuracy'])

In [0]:
# setup early stopping

cb = EarlyStopping(min_delta=0.0001, patience=3)

In [38]:
my_history = my_model.fit(x=x_train, y=y_train, epochs=100, validation_data=(x_test, y_test), callbacks=cb)

Epoch 1/100
11/11 [==============================] - 0s 43ms/step - loss: 0.0016 - accuracy: 1.0000 - val_loss: 0.4094 - val_accuracy: 0.9373
Epoch 2/100
11/11 [==============================] - 0s 43ms/step - loss: 0.0012 - accuracy: 1.0000 - val_loss: 0.4215 - val_accuracy: 0.9288
Epoch 3/100
11/11 [==============================] - 0s 43ms/step - loss: 8.8220e-04 - accuracy: 1.0000 - val_loss: 0.4449 - val_accuracy: 0.9288
Epoch 4/100
11/11 [==============================] - 0s 42ms/step - loss: 7.2769e-04 - accuracy: 1.0000 - val_loss: 0.5456 - val_accuracy: 0.9288


# Custom CNN Model with Image Manipulations
## *This a stretch goal, and it's relatively difficult*

To simulate an increase in a sample of image, you can apply image manipulation techniques: cropping, rotation, stretching, etc. Luckily Keras has some handy functions for us to apply these techniques to our mountain and forest example. Check out these resources to help you get started: 

1. [Keras `ImageGenerator` Class](https://keras.io/preprocessing/image/#imagedatagenerator-class)
2. [Building a powerful image classifier with very little data](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html)
 

In [0]:
# State Code for Image Manipulation Here

# Resources and Stretch Goals

Stretch goals
- Enhance your code to use classes/functions and accept terms to search and classes to look for in recognizing the downloaded images (e.g. download images of parties, recognize all that contain balloons)
- Check out [other available pretrained networks](https://tfhub.dev), try some and compare
- Image recognition/classification is somewhat solved, but *relationships* between entities and describing an image is not - check out some of the extended resources (e.g. [Visual Genome](https://visualgenome.org/)) on the topic
- Transfer learning - using images you source yourself, [retrain a classifier](https://www.tensorflow.org/hub/tutorials/image_retraining) with a new category
- (Not CNN related) Use [piexif](https://pypi.org/project/piexif/) to check out the metadata of images passed in to your system - see if they're from a national park! (Note - many images lack GPS metadata, so this won't work in most cases, but still cool)

Resources
- [Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385) - influential paper (introduced ResNet)
- [YOLO: Real-Time Object Detection](https://pjreddie.com/darknet/yolo/) - an influential convolution based object detection system, focused on inference speed (for applications to e.g. self driving vehicles)
- [R-CNN, Fast R-CNN, Faster R-CNN, YOLO](https://towardsdatascience.com/r-cnn-fast-r-cnn-faster-r-cnn-yolo-object-detection-algorithms-36d53571365e) - comparison of object detection systems
- [Common Objects in Context](http://cocodataset.org/) - a large-scale object detection, segmentation, and captioning dataset
- [Visual Genome](https://visualgenome.org/) - a dataset, a knowledge base, an ongoing effort to connect structured image concepts to language